In [1]:
!python --version
%pip install --upgrade pandas
%pip install openpyxl
%pip install numpy
%pip install scikit-learn
%pip install matplotlib
%pip install plot_keras_history
%pip install tensorflow==2.1
%pip install keras-tuner

Python 3.6.13 :: Anaconda, Inc.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import time
import datetime

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from plot_keras_history import plot_history
import sklearn.metrics as metrics
import os as os

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.python.client import device_lib
# import keras_tuner as kerastuner
# from keras_tuner.tuners import RandomSearch


print("TF version:", tf.__version__)
print("Panda version:", pd.__version__)


def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

# Se va a habilitar la dedicacion dinamica de memoria para que la GPU vaya asignando recursos al proceso conforme los vaya necesitando

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
#print(tf.config.list_physical_devices('GPU'))
print("Build with CUDA: ", tf.test.is_built_with_cuda())



# Historico de datos para realizar la predicción
W = 730
# Insantes de tiempo futuros a predecir
H = 10

# Porcentaje del conjunto de test
test_size = 0.3
# Porcentaje del conjunto de validacion
val_size = 0.3


# Mejores modelos que se quieren obtener
numBestModels = 1
# Establecer iteraciones (max_trials) y ejecuciones por cada iteracion (executions_per_trial)
# max_trials = Total number of trials (model configurations) to test at most. Note that the oracle may interrupt the search before max_trial models have been tested if the search space has been exhausted.
trials = 10
executions = 20
# Establecer objetivo (Name of model metric to minimize or maximize, e.g. "val_accuracy"). el "val_" hace referencia a que se coge la métrica en el subconjunto de validación

# obje = kerastuner.Objective('val_mean_absolute_percentage_error', 'min')


# Epocas
searchEpochs = 2
epch = 10
# Tamaño del batch
batch = 1024
# Establecer minimo y maximo de unidades, el salto en el que pueden cambiar y el valor por defecto
minUnits = 50
maxUnits = 300
stepsUnits = 25
defaultUnits = 50
# Establecer minimo y maximo de capas y el valor por defecto
minLayers = 1
maxLayers = 10
defaultLayers = 3
# Establecer Dropout
minDropout = 0
maxDropout = .4
defaultDropout = .2
# Establecer learning rate
lr = [0.0, 1e-2, 1e-3, 1e-4]

TF version: 2.1.0
Panda version: 1.1.5
Default GPU Device: /device:GPU:0
Num GPUs Available:  1
Build with CUDA:  True


In [4]:
df = pd.read_excel("C:/Users/hecto/Documents/Master/TFM/tfm-renewable-energy-deep-learning/2015_30min.xlsx", header=None, engine='openpyxl')[0]

# Data preprocessing: each row will contain the 10 measures for each day , and the 10 measures for the following day
X = pd.DataFrame(np.array(df).reshape(-1, 10), columns=["col_{}".format(i) for i in range(0, 10)])

Y = pd.DataFrame.copy(X)

Y.columns = ["col_{}".format(i) for i in range(11, 21)]
Y = Y.drop(0)
Y = Y.reset_index(drop=True)
Y.loc[len(Y)] = np.zeros(10)

dfPreproccessed = pd.concat([X, Y], axis=1)

print("DataFrame Preproccessed:")
print(dfPreproccessed)
print("---------------------------------------------")

pd.set_option('display.max_columns', dfPreproccessed.shape[1])
print("Head DataFrame Preprocessed\n " + str(dfPreproccessed.head()))
print("---------------------------------------------")

# Split the data into training and validation sets
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size = test_size, random_state = 0, shuffle=False)
xTrain, xVal, yTrain, yVal = train_test_split(xTrain, yTrain, test_size = val_size, random_state = 0, shuffle=False)

# Show subsets shapes
print("Shapes:")
print("xTrain:\t"+str(xTrain.shape))
print("yTrain:\t"+str(yTrain.shape))
print("xVal:\t"+str(xVal.shape))
print("yVal:\t"+str(yVal.shape))
print("xTest:\t"+str(xTest.shape))
print("yTest:\t"+str(yTest.shape))

print("---------------------------------------------")

# Adding the third dimension needed to use LSTM (samples, timestamps, features)
xtrain = np.reshape(xTrain.values, (xTrain.shape[0], xTrain.shape[1], 1))
ytrain = np.reshape(yTrain.values, (yTrain.shape[0], yTrain.shape[1], 1))
xval = np.reshape(xVal.values, (xVal.shape[0], xVal.shape[1], 1))
yval = np.reshape(yVal.values, (yVal.shape[0], yVal.shape[1], 1))
xtest = np.reshape(xTest.values, (xTest.shape[0], xTest.shape[1], 1))
ytest = np.reshape(yTest.values, (yTest.shape[0], yTest.shape[1], 1))

# New shapes after third dimension added
print("Dimensions afther third dimension added:")
print("xTrain:\t"+str(xtrain.shape))
print("yTrain:\t"+str(ytrain.shape))
print("xVal:\t"+str(xval.shape))
print("yVal:\t"+str(yval.shape))
print("xTest:\t"+str(xtest.shape))
print("yTest:\t"+str(ytest.shape))

DataFrame Preproccessed:
          col_0       col_1       col_2       col_3       col_4       col_5  \
0     96.691833  193.486833  700.986333  587.718333  454.512500  631.489833   
1    849.077500  746.732500  850.424667  745.088500  755.706667  688.645000   
2    202.375667  166.977000  403.916667  340.241000  649.179667  708.682500   
3    649.432333  665.403833  703.733167  438.760333  198.212667  114.285000   
4    109.810167  152.215167   84.124833  134.679500  183.520167  280.223333   
..          ...         ...         ...         ...         ...         ...   
725  360.369333  166.942500  390.007500  377.754500  226.036833  166.183833   
726  363.149000  464.485667  546.436500  617.035333  419.664667  571.180833   
727  571.733833  562.752667  548.686000  715.050000  446.255333  370.894167   
728  323.271333  320.158667  281.495667  465.665167  391.321167  609.922000   
729  467.921333  417.395833  578.181500  655.602167  524.690833  601.943333   

          col_6       col_

In [5]:
# Model definition
model = Sequential()
model.add(LSTM(32, input_shape=(xtrain.shape[1], 1)))
model.add(Dense(1, activation="sigmoid"))

# Model compilation
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(
    xtrain.values.reshape((xtrain.shape[0], xtrain.shape[1], 1)),
    ytrain,
    epochs=1,
    batch_size=16,
    validation_data=(xval.values.reshape((xval.shape[0], xval.shape[1], 1)), yval))

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [ ]:
# Training and Validation loss curves
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()
plt.show()

# Model evaluation with validation data
score = model.evaluate(xval.reshape((xval.shape[0], xval.shape[1], 1)), yval)
print('Validation loss:', score)